# The NonRandomTwoLiquids (NRTL) model for *excess Gibbs energy* ($g^E$) and a case study of the Liquid-Liquid equilibria of water+ethanol+limonene

# The NonRandomTwoLiquids (NRTL) model

Correlates excess Gibbs energy and its derivatives: e.g. activity coefficients at given T, P and composition for a liquid mixture.

## References:
* Renon H., Prausnitz J. M., Local Compositions in Thermodynamic Excess Functions for Liquid Mixtures, AIChE J., 14_1, S.135–144, 1968.pdf
* Prausnitz, Lichtenthaler & Azevedo, Molecular Thermodynamic of Fluid Phase Equilibria, 1998

# Model overview

|--------------Renon & Prausnitz--------------|
|:-:|
|$ \frac{g^E}{RT}=\sum_{i=1}^n \left[ x_i\frac{\sum_{j=1}^n \tau_{j,i} G_{j,i} x_{j}}{\sum_{k=1}^n G_{k,i} x_k} \right] $

Where


|--------------Renon & Prausnitz--------------|
|:-:|
|$\tau_{i,j}= \frac{g_{i,j}-g_{j,j}}{RT}=\frac{A_{i,j}}{T}$ |
|$G_{i,j}=\mathrm{exp}(-\alpha_{i,j} \tau_{i,j})$|

And either each $(g_{i,j}-g_{j,j})$ difference, in units of energy, or  each $A_{i,j}$ binary parameter, in units of temperature, are usually fitted to experimental data and published.

The activities coefficients are calculated from the derivative of excess gibbs energy with respect to mole number of componente i ($n_i=N x_i$) with T, P and mole number of every other component held constant, ($RTln(\gamma_i) = {\partial g^E}/{ \partial n_i}$)therefore:


|---------------------------------------Renon & Prausnitz--------------------------------------|
|:-:|:-:|
|$ln(\gamma_i)=  \frac{\sum_{j=1}^n\left[\tau_{j,i} G_{j,i} x_{j}\right]}{\sum_{k=1}^n\left[G_{k,i}x_{k}\right]} + \sum_{j=1}^n\left[ \left(\frac{\ G_{i,j} x_{j}}{\sum_{k=1}^n\left[G_{k,j}x_{k}\right]}\right) \left(\tau_{i,j}-\frac{\sum_{j=1}^n\left[\tau_{i,j} G_{i,j} x_{i}\right]}{\sum_{k=1}^n\left[G_{k,j}x_{k}\right]} \right) \right] $|

# Gather the needed packages here

In [12]:
import numpy as np

# Model inputs

Experimental parameters from Cháfer et al., 2004

Ref: Cháfer, Muñoz, Burguet, Berna, The influence of the temperature on the liquid–liquid equlibria of the mixture limonene + ethanol + H 2 O, Fluid Phase Equilibria 224 (2004) 251–256

### Thermodynamics *degrees of freedom*
Excess gibbs energy, and activity coefficients are natural functions of T, P and composition.
Here, according to the published parameters, the validity of the model is for constant pressure of 1 atm.
The remaining D.o.F are T and x


In [13]:
#trial temperature and composition:
T = 293.15 #K
x=np.array([.1,.3,.6])

## Fitted parameters

In [14]:
alpha=0.2

A12=107.99 #K
A21=555.81 #K
A13=1011.98 #K
A31=2277.37 #K
A23=-1113.1 #K
A32=1217.37 #K

## Feeding the fitted parameters to the model in conventional notation:

In [15]:
#assemble matrix
tau = np.array([[0, A12, A13],
             [A21, 0, A23],
             [A31, A32, 0]])/(T)

# one value for alpha feeds all three binary alphas
alpha12=alpha13=alpha23=alpha

#matrix is symmetric
alpha21=alpha12
alpha31=alpha13
alpha32=alpha23

#assemble matrix
alpha = np.array([[0, alpha12, alpha13],
                [alpha21, 0, alpha23],
                [alpha31, alpha32, 0]])

#verify
print("tau=")
print(tau)
print("alpha=")
print(alpha)

tau=
[[ 0.          0.36837796  3.45208937]
 [ 1.89599181  0.         -3.79703224]
 [ 7.76861675  4.15272045  0.        ]]
alpha=
[[ 0.   0.2  0.2]
 [ 0.2  0.   0.2]
 [ 0.2  0.2  0. ]]


# Model equations
(try them in a script-wise manner, consider input fitted parameters and thermodynamic D.o.F. provided

In [16]:
G=np.zeros([3,3])
for j in range(3):
    for i in range(3):
        G[j,i]=np.exp((-alpha[j,i]*tau[j,i]))
print("G=")
print(G)

G=
[[ 1.          0.92897301  0.50136652]
 [ 0.68440984  1.          2.13700742]
 [ 0.21145917  0.4358121   1.        ]]


In [17]:
Gamma=np.zeros([3])
for i in range(3):

    Sj1=0
    Sj2=0
    Sj3=0
    for j in range(3):
        Sj1     += tau[j,i]*G[j,i]*x[j]
        Sj2     += G[j,i]*x[j]

        Sk1=0
        Sk2=0
        Sk3=0
        for k in range(3):
            Sk1+=G[k,j]*x[k]
            Sk2+=x[k]*tau[k,j]*G[k,j]
            Sk3+=G[k,j]*x[k]
        
        Sj3     += ((x[j]*G[i,j])/(Sk1))*(tau[i,j]-(Sk2)/(Sk3))
    
    Gamma[i]=np.exp(Sj1/Sj2 + Sj3)
    
print(Gamma)

[ 21.87429222   0.27033844   0.79826439]


# Liq-Liq Equilibria Flash
The next step will be using the model in a phase equilibria algorithm, a LiqLiq equilibria flash

it works as follows:

D.o.F for a flash calculation: T, P, global composition - z



* ...............  =>guess, xL1 xL2, then calls model(T,P,x)
* ...............//
* .............<=
* T,P,z => algorithm => finds equilibrium xL1 xL2 and BETA

**Equilibriuma criteria**

mu1[:]=mu2[:]

**Devised algorithm after analytical simplification of repeated contributions:**

x1*gamma1[:]=x2*gamma2[:]

The first preliminary step is to functionize the model for recursive usage:


## Functionize the model

$$\underline{\gamma} \leftarrow {\gamma} \left(\{T,\underline {x}\},\{ \underline {\underline{\alpha}}, \underline {\underline{\tau}}\}\right)$$

Easyest way to functionalize the model is copy the expressions already presented in the scripwise approach
and the wrap in a def block:
We propose the use of tuples to separate input from thermodynamics dof and experimental parameters conceptually
this is not mandatory for the wrapping concept, see later

In [18]:
def Gamma(ThermoDOF,ExpPARAM):
    (T,x)=ThermoDOF
    (alpha,tau)=ExpPARAM

    G=np.zeros([3,3])
    for j in range(3):
        for i in range(3):
            G[j,i]=np.exp((-alpha[j,i]*tau[j,i]))
    
    Gamma=np.zeros([3])
    for i in range(3):

        Sj1=0
        Sj2=0
        Sj3=0
        for j in range(3):
            Sj1 += tau[j,i]*G[j,i]*x[j]
            Sj2 += G[j,i]*x[j]
    
            Sk1=0
            Sk2=0
            Sk3=0
            for k in range(3):
                Sk1 += G[k,j]*x[k]
                Sk2 += x[k]*tau[k,j]*G[k,j]
                Sk3 += G[k,j]*x[k]
            
            Sj3 += ((x[j]*G[i,j])/(Sk1))*(tau[i,j]-(Sk2)/(Sk3))
        
        Gamma[i]=np.exp(Sj1/Sj2 + Sj3)
    
    return Gamma
  
print(Gamma((T,x),(alpha,tau))) #ttest using those trial input

[ 21.87429222   0.27033844   0.79826439]


# Applying linear algebra matrix notation

These calculations can be rewritten using linear algebra
which can improve readability, maintainability, reduce human error in the coding, and improve execution speed,
See Lectures notes of Abreu, C. R. A. (Matrix Algebra and Matrix Differentiation Rules Applied to Excess Gibbs Energy Models) in our library [here](absolute url pending)

|--------------Renon & Prausnitz--------------|-------------------Abreu-------------------|
|:-:|:-:|
|$ \frac{g^E}{RT}=\sum_{i=1}^n \left[ x_i\frac{\sum_{j=1}^n \tau_{j,i} G_{j,i} x_{j}}{\sum_{k=1}^n G_{k,i} x_k} \right] $|$ \frac{g^E}{RT}=\underline{x}^T\underline{\underline{E}} \underline{x}$ |

Where

|--------------Renon & Prausnitz--------------|-------------------Abreu-------------------|
|:-:|:-:|
|$\tau_{i,j}= \frac{g_{i,j}-g_{j,j}}{RT}=\frac{A_{i,j}}{T}$ | $\underline{\underline{\tau}}=-T^{-1}\underline{\underline{A}}$|
|$G_{i,j}=\mathrm{exp}(-\alpha_{i,j} \tau_{i,j})$|$\underline{\underline{G}}= \mathrm{exp}(\underline{\underline{\alpha}} \circ \underline{\underline{\tau}})$|
|---|$\underline{\underline{\Lambda}}=(\underline{\underline{\tau}} \circ \underline{\underline{G}})$|
|---|$\underline{\underline{E}}=\underline{\underline{\Lambda}}\mathscr{D}^{-1}(\underline{\underline{G}}^T\underline{x})$|

And either each $(g_{i,j}-g_{j,j})$ difference, in units of energy, or  each $A_{i,j}$ binary parameter, in units of temperature, are usually fitted to experimental data and published.

therefore


|---------------------------------------Renon & Prausnitz--------------------------------------|-------------------Abreu-------------------|
|:-:|:-:|
|$ln(\gamma_i)=  \frac{\sum_{j=1}^n\left[\tau_{j,i} G_{j,i} x_{j}\right]}{\sum_{k=1}^n\left[G_{k,i}x_{k}\right]} + \sum_{j=1}^n\left[ \left(\frac{\ G_{i,j} x_{j}}{\sum_{k=1}^n\left[G_{k,j}x_{k}\right]}\right) \left(\tau_{i,j}-\frac{\sum_{j=1}^n\left[\tau_{i,j} G_{i,j} x_{i}\right]}{\sum_{k=1}^n\left[G_{k,j}x_{k}\right]} \right) \right] $|$ ln(\underline{\gamma})=\left(\underline{\underline{E}}^S-\underline{\underline{L}}\mathscr{D}\underline{\underline{E}}^T \right)\underline{x}$|

where

* the symbol $\underline{\underline{M}} \circ \underline{\underline{N}}$ is the Hadamard product, element-wise multiplication between matrices $\underline{\underline{M}} $and$ \underline{\underline{N}}$.

* $\mathscr{D}\underline{v}$ means matrix diagonalization of an array $\underline{v}$: $\mathscr{D}\underline{v}= \left\{ \underline{\underline{M}} \mid M_{i,i}=v_{i},M_{i,j \neq i}=0 \right\}$.

* $\underline{\underline{M}}^S$ means symmetrization of a matrix $\underline{\underline{M}}$:  $\underline{\underline{M}}^S= \left\{ \underline{\underline{N}} \mid N_{i,j}=M_{i,j}+M_{j,i}\right\}$
* $\underline{\underline{M}}^T$ means transposition of a matrix $\underline{\underline{M}}$: $\underline{\underline{M}}^T= \left\{ \underline{\underline{N}} \mid N_{i,j}=M_{j,i} \right\}$

Diagonalization is used to represent element wise multiplication between lines or columns.
See:
if
$\underline{v3}= \underline{v1} \mathscr D  \underline{v2}$
then
$v3_i= v1_i * v2_i$

This is useful in analytical differentiation

This may be dropped at implementation time, depending on the programming enviroment
See below:




matrix notation: v1 o v2 = matriz operators notation D v1 v2 = final python implement: v1*v2 (defaults termo-a-termo)
matriz operators notation: E D v1 = [ E[1,:] o v1[1]; E[2,:] o v1[2]; E[3,:] o v1[3] ]= final python implement: E * v1.T

see
* **matrix multiplication**

conventional matrix notation

$\underline{\underline{C}}=\underline{\underline{A}}\underline{\underline{B}}$

elemental definition:

$C_{i,j} = \sum_k{A_{i,k}*B_{k,j}}$

python implementation

`C = A @ B`

* **element-wise multiplication**

conventional matrix notation

$\underline{\underline{C}}=\underline{\underline{A}} \circ \underline{\underline{B}}$

elemental definition:

$C_{i,j} = A_{i,j}*B_{i,j}$

python implementation

`C = A * B`

* **element wise multiplication with broadcasting -- matmul+diag equivalence**

conventional matrix notation

$\underline{\underline{C}}=\underline{\underline{A}} \mathscr D \underline{b}$

$\underline{\underline{D}}= \mathscr D \underline{b} \underline{\underline{A}}$

elemental definition:

$C_{i,j} = A_{i,j}*b_{j}$

$D_{i,j} = b_{i}*A_{i,j}$

python implementation

`C = A * b.T`

`D = b * A

## Use linear algebra in the model

In [22]:
def linalgGamma(ThermoDOF,ExpPARAM):
    T=ThermoDOF[0] #should be an scalar and a 
    x=ThermoDOF[1] #should be a single-column matrix, not quite the same as a 1d array, see the usage test below
    alpha=ExpPARAM[0] #should be two matrix
    tau=ExpPARAM[1]
    
    G=np.exp((-alpha*tau)) #M2d * N2d yields element-wise multiplication
    Lambda=(tau*G) 
    den=G.T @ x #M.T yields the transpose matrix of M; M @ N yields the matrix multiplication between M and N
    invden=1/den #scalar 1 is broadcast for element-wise division
    E0 = Lambda * invden.T
    L0 = G * invden.T
    term2 = L0 * x.T
    term3 = term2 @ E0.T
    term4 = (E0+E0.T-term3) @ x
    gamma = np.exp(term4)
    return gamma
  
x_as_column = np.array([x]).T #we wrap x with an extra braket so it is now a 2d array, a matrix, as we did not add any extra lines it is a single-line matrix, we tranpose to generate a single-column matrix (1d arrays cannot be transposed, there is no second dimension)
print(linalgGamma((T,x_as_column),(alpha,tau))) #test using those trial input

[[ 21.87429222]
 [  0.27033844]
 [  0.79826439]]


Ipython provides a profiling tool, with %timeit you can evaluate the time for execution of a line of program (with all called dependencies). we use it to evaluate our function in version 1 with explicit for loops and in version 2 with linear algebra matrix operations

In [13]:
%timeit Gamma((T,x),(alpha,tau)) #ttest using those trial input #My result was 131 micro seconds per loop

10000 loops, best of 3: 97 µs per loop


In [12]:
%timeit linalgGamma((T,x_as_column),(alpha,tau)) #ttest using those trial input #My result was 39.9 micro seconds per loop

The slowest run took 269.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 27 µs per loop


# Coming soon:
* some plots
* Liq Liq Equilibria flash algorithm
* more plots
* more optimization with fortran, c, cython and numba